# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve Examples 3.38 of [BPT12] with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

[BPT12] Blekherman, G. & Parrilo, P. A. & Thomas, R. R.
*Semidefinite Optimization and Convex Algebraic Geometry*.
Society for Industrial and Applied Mathematics, **2012**.

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

Iter: 10 Ap: 9.69e-01 Pobj: -1.0000000e+00 Ad: 9.00e-01 Dobj: -1.0000000e+00 
Success: SDP solved
Primal objective value: -1.0000000e+00 
Dual objective value: -1.0000000e+00 
Relative primal infeasibility: 8.36e-16 
Relative dual infeasibility: 1.91e-09 
Real Relative Gap: -1.02e-09 
XZ Relative Gap: 3.61e-09 
DIMACS error measures: 1.60e-15 0.00e+00 4.55e-09 0.00e+00 -1.02e-09 3.61e-09
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3401580e+01 


* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.46225e-01


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.8025916994817284*x^2 - 0.12696358959695092*x*y + 2.1931536335606583*y^2 - 0.4175974829737728*x - 0.09754582659048598*y - 0.6797576582812243)^2 + (-0.9300185097227982*x^2 - 1.229323003076067*x*y - 0.1077026229314299*y^2 + 0.2742228250653725*x - 0.6455024395913428*y + 0.9043624572847013)^2 + (0.44438911573273826*x^2 - 0.15182100493580486*x*y - 0.20409230951833654*y^2 - 0.6219741186490512*x - 1.3853650445041532*y - 0.5739128592301717)^2 + (0.020943538499585978*x^2 - 0.7618732513464348*x*y - 0.2555835927433955*y^2 - 0.34368939804846743*x + 0.51841980639472*y - 0.5702898506776773)^2 + (-0.5053890156943566*x^2 + 0.2972255843408651*x*y - 0.24791041664865923*y^2 - 0.4999809437701398*x + 0.049133727851402645*y + 0.04145089656627004)^2 + (-0.19383952209211255*x^2 + 0.05960910807713195*x*y - 0.10020512085870777*y^2 + 0.25390202282612473*x - 0.06335690502229822*y - 0.25245477072168304)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  159.103 μs (3305 allocations: 449.61 KiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  408.207 μs (5901 allocations: 566.66 KiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.65e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 2.05e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.90e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9858991e-02 
Iter:  3 Ap: 7.78e-01 Pobj:  0.0000000e+00 Ad: 7.55e-01 Dobj:  5.6667085e-01 
Iter:  4 Ap: 7.59e-01 Pobj:  0.0000000e+00 Ad: 8.08e-01 Dobj:  4.9238811e-01 
Iter:  5 Ap: 7.15e-01 Pobj:  0.0000000e+00 Ad: 6.42e-01 Dobj:  4.3470711e-01 
Iter:  6 Ap: 6.45e-01 Pobj:  0.0000000e+00 Ad: 7.00e-01 Dobj:  2.1752403e-01 
Iter:  7 Ap: 6.50e-01 Pobj:  0.0000000e+00 Ad: 6.29e-01 Dobj:  1.2996035e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 9.84602e-02


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-24.927154511803934*x^2 + 27.80018416438059*x*y - 6.940895093277731*y^2 - 0.00025745451936590787*x + 0.0004423818427874241*y + 7.557893351745502e-5)^2 + (-21.341702974684384*x^2 - 13.61274013419524*x*y + 22.122686306231827*y^2 - 0.0006202089859727377*x - 0.0004698520241215514*y - 1.6865925877058706e-5)^2 + (-0.07015469214184664*x^2 - 0.09428679976390739*x*y - 0.12569540965407205*y^2 + 0.00851835392876935*x - 0.006590600623452223*y + 4.623162208288533e-5)^2 + (-0.0017527827110082304*x^2 - 0.002357290371187502*x*y - 0.0031420558911365304*y^2 - 0.054529602188287424*x + 0.04182710745418746*y - 2.3524778609698603e-5)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

-7.953798625403435e-6x² - 2.076585154972988e-5xy - 1.3525048331121782e-5y² - 1.8162242906209093e-7x - 2.3916558583771576e-7y + 8.687412736376387e-9

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

0.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*